# Fase de entrenamiento del modelo

Recordar correr el colab en GPU

In [ ]:
import os
import pandas as pd
import json
import pickle
import zipfile
import shutil

In [ ]:
# Instalamos los paquetes necesarios para que funcione desde la Colab
!pip install avro-python3

!pip install tf_slim==1.1.0
!pip install tf-models-official==2.7.0
!pip install lvis
!pip install tensorflow_io==0.23.1
!pip install keras==2.7.0
!pip install opencv-python-headless==4.5.2.52

  Preparing metadata (setup.py) ... done
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=43992 sha256=b76837225d1f78c5e3e13b56f566c3adf8e56b8fd1b25d37a6b9db9f80b7a9d4
  Stored in directory: /root/.cache/pip/wheels/bc/85/62/6cdd81c56f923946b401cecff38055b94c9b766927f7d8ca82
Successfully built avro-python3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.

Aprendizaje por transferencia es tomar una red ya entrenada en miles de imagenes y  modificar la ultima capa para que clasifique en tipos de objetos

In [ ]:
labels = [
    {'name': 'carros', 'id': 1},
    {'name': 'motos', 'id': 2},
  ]

In [ ]:
with open("label_map.pbtxt", "w") as f:
  for label in labels:
    f.write('item { \n')
    f.write('\tname:\'{}\'\n'.format(label['name']))
    f.write('\tid:{}\n'.format(label['id']))
    f.write('}\n')

In [ ]:
!rm -rf /content/models/

In [ ]:
import os
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/

# !git checkout 58d19c67e1d30d905dd5c6e5092348658fed80af

!apt-get update && apt-get install -y -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

/content
/content/models
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
E: Unable to locate package python-pil
E: Unable to locate package python-lxml
/content/models/research
2024-04-08 01:30:48.936540: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is

## TensorFlow 2 Detection Model Zoo


https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

Vamos a usar:

SSD MobileNet V2 FPNLite 320x320

http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

In [ ]:
!wget --no-check-certificate http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz \
-O /content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

--2024-04-08 01:31:28--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.195.207, 172.253.117.207, 142.250.99.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.195.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

/content/ssd_mobile 100%[===================>]  19.56M   106MB/s    in 0.2s    

2024-04-08 01:31:28 (106 MB/s) - ‘/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]



In [ ]:
!tar -zxvf /content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/saved_model.pb
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.index


In [ ]:
output_path = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
output_path = os.path.join(os.getcwd(), output_path)
print("La carpeta se almaceno en {}".format(output_path))

La carpeta se almaceno en /content/models/research/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8


# Balanceo de imágenes y data augmentation

In [ ]:
path_training = '/content/ssd_mobilenet'

os.mkdir(path_training)


In [ ]:
source_config = f'{output_path}/pipeline.config'
target_config = f"{path_training}/pipeline.config"
shutil.copyfile(source_config, target_config)

'/content/ssd_mobilenet/pipeline.config'

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

TypeError: api_export.__init__() got an unexpected keyword argument 'metaclass'

In [ ]:

# Obtenemos la configuración del archivo pipeline
config = config_util.get_configs_from_pipeline_file(target_config)
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()

# hemos dejado el archivo listo para ser editable
with tf.io.gfile.GFile(target_config, "r") as f:
  proto_str = f.read()
  # todo esto es solo para editar el archivo de configuracion
  text_format.Merge(proto_str, pipeline_config)

In [ ]:
pipeline_config

model {
  ssd {
    num_classes: 90
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.009999999776482582
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0


In [ ]:
# Definimos las rutas en donde se encuentra los TFRecords y el label map para agregarlos al archivo de configuración del pipeline.config
label_map_pbtxt_fname = '/content/label_map.pbtxt'

train_record_fname = "/content/dataset_train.record"
test_record_fname = "/content/dataset_test.record"

### Super importante

Es muy importante editar estas configuraciones del archivo

ya que aqui estamos configurando para que ocurra un data augmentation

In [ ]:
# Cantidad de clases del modelo
pipeline_config.model.ssd.num_classes = 2

# El tamaño del batch size, entre más grande mas costo computacional va a necesitar en el proceso de entrenamiento, pero a su vez entrenara más rapido.
pipeline_config.train_config.batch_size = 4

# Checkpoint
# si estamos entrenando un modelo muy complejo a lo largo de dias
# podemos guardar y seleccionar desde que checkpoint
# queremos hacer el entrenamiento
# Donde almacenaremos los resultados del entrenamiento
pipeline_config.train_config.fine_tune_checkpoint =f"{output_path}/checkpoint/ckpt-0"

# Qué tipo de deteción aplicaremos (Object detection)
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"

# Dirección del label map, archivo donde se contienen las clases
pipeline_config.train_input_reader.label_map_path = label_map_pbtxt_fname

# Dirección del train TFRecord
pipeline_config.train_input_reader.tf_record_input_reader.input_path[0] = train_record_fname

# Dirección del label map
pipeline_config.eval_input_reader[0].label_map_path = label_map_pbtxt_fname

# Dirección del test TFRecord
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[0] = test_record_fname


In [ ]:
# Visualizamos nuestro pipeline_config final.
pipeline_config

model {
  ssd {
    num_classes: 2
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.009999999776482582
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
 

# Entrena, evalua y optimiza con TensorBoard

In [ ]:

# guardamos el objeto en un archivo
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(target_config, "wb") as f:
  f.write(config_text)


In [ ]:
num_steps = 5000
# ruta donde se va a guardar el modelo
model_dir = "/content/ssd_mobilenet"

In [ ]:
target_config

'/content/ssd_mobilenet/pipeline.config'

In [ ]:
# Utilizamos directamente la libreria de object detection para ejecutar el script model_main_tf2
# Los argumentos principales son el archivo de configuración, la ubicación del modelo y la cantidad de steps que ejecutará.
# pipeline_config_path es de donde viene el archivo
# model_dir direccion donde vamos a guardar el modelo
# num_train_steps numero de pasos
!python /content/models/research/object_detection/model_main_tf2.py \
--pipeline_config_path={target_config} \
--model_dir={model_dir} \
--num_train_steps={num_steps}

2024-04-08 00:47:03.739251: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2024-04-08 00:47:03.739284: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Traceback (most recent call last):
  File "/content/models/research/object_detection/model_main_tf2.py", line 31, in <module>
    from object_detection import model_lib_v2
ModuleNotFoundError: No module named 'object_detection'
